By default, Jupyter notebooks do not have intellisense. If you like to enable it, add following code.

In [230]:
# enable intellisense
%config IPCompleter.greedy=True

# Binary addition
_What exactly will the RNN learn?_

**RNN is going to learn the carry bit on its own!**


| input1 | input2 | carry-in | sum | carry-out |
|:---:|:---:|:---:|:---:|:---:|
| 0 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 | 0 |
| 0 | 1 | 0 | 1 | 0 |
| 0 | 1 | 1 | 0 | 1 |
| 1 | 0 | 0 | 1 | 0 |
| 1 | 0 | 1 | 0 | 1 |
| 1 | 1 | 0 | 0 | 1 |
| 1 | 1 | 1 | 1 | 1 |

## Samples
The first step, sample data is needed.
One looup table is used to help us converting int to binary and vice versa

int2binary (__lookup table__)

| int | binary array |
| :--- | :---: |
| 0 | [0, 0, 0, 0, 0, 0, 0, 0] |
| 1 | [0, 0, 0, 0, 0, 0, 0, 1] |
| 2 | [0, 0, 0, 0, 0, 0, 1, 0] |
...
| 255 | [1, 1, 1, 1, 1, 1, 1, 1] |

In [248]:
import copy, numpy as np
from abc import ABC, abstractmethod

It's good practice to seed your random numbers. Your numbers will still be randomly distributed, but they'll be randomly distributed in exactly the same way each time you train. This makes it easier to see how your changes affect the network.

In [249]:
np.random.seed(0)

In [250]:
class dataset:
    def __init__(self, binary_dim):
        # creating lookup table for converting int to binary
        self.int2binary = {}
        
        self.largest_number = pow(2,binary_dim)
        range_numbers = range(self.largest_number)
        
        # genrating corresponding binary array
        # for example binary[0] = array([0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)
        binary = np.unpackbits(np.array([range_numbers],dtype=np.uint8).T,axis=1)
        
        # adding binary array to int2binary (lookup table)
        for i in range_numbers:
            self.int2binary[i] = binary[i]
    
    # generate a sample addition problem (a + b = c)
    def get_sample_addition_problem(self):
        a_int = np.random.randint(self.largest_number/2) # int version # generate random int between [1,largest_number/2)
        a = self.int2binary[a_int] # binary encoding

        b_int = np.random.randint(self.largest_number/2) # int version
        b = self.int2binary[b_int] # binary encoding

        # true answer => summation
        c_int = a_int + b_int
        c = self.int2binary[c_int]

        return a, b, c, a_int, b_int, c_int


In [251]:
class activation(ABC):
    
    @abstractmethod
    def forward(self, net):
        pass
    
    @abstractmethod
    def backward(self, output):
        pass

**sigmoid activation function**

A sigmoid function maps any value to a value between 0 and 1. 

forward

$$ \sigma(x) = \frac{1}{1+e^{-x}}$$

backward
$$ \frac{\partial \sigma(x)}{\partial x} =  \sigma(x)(1- \sigma(x))$$

In [252]:
class sigmoid_activation(activation):
        
    def forward(self, net):
        return 1/(1 + np.exp(-net))
    
    def backward(self, output):
        return output*(1 - output)

In every layer, number of neurons along with activation function should be defined.

In [253]:
class network_layer(ABC):
    
    def __init__(self, neuron_count):
        self.neuron_count = neuron_count

In [256]:
class input_layer(network_layer): 
    
    def forward(self, X, W_input):
        return np.dot(X,W_input)

In [257]:
class hiddenLayerUnfold:
    
    def __init__(self, neuron_count):
        
        # Save the values obtained at Hidden Layer of current state in a list to keep track
        self.hidden_layer_values  = list()
        
        # Initially, there is no previous hidden state. So append "0" for that
        self.hidden_layer_values.append(np.zeros(neuron_count))
    
    def save_previous_hidden_layer_value(self, previous_hidden_layer_value):
        self.hidden_layer_values.append(copy.deepcopy(previous_hidden_layer_value))

In [258]:
class hidden_layer(network_layer):
    
    def __init__(self, neuron_count):
        super().__init__(neuron_count)
        #self.hiddenLayerUnfold = hiddenLayerUnfold(neuron_count)
        # Save the values obtained at Hidden Layer of current state in a list to keep track
        self.hidden_layer_values  = list()
        
        # Initially, there is no previous hidden state. So append "0" for that
        self.hidden_layer_values.append(np.zeros(neuron_count))
    
    def forward(self, input_layer_output, W_hidden):
        prev_hidden = self.hidden_layer_values[-1]
        
        print("input_layer_output", input_layer_output)
        print("prev_hidden", prev_hidden)
        print(np.dot(prev_hidden, W_hidden))
        
        net_hidden = input_layer_output + np.dot(prev_hidden, W_hidden)
        sigmoid = sigmoid_activation()
        return sigmoid.forward(net_hidden)
    
    def save_previous_hidden_layer_value(self, previous_hidden_layer_value):
        #self.hiddenLayerUnfold.save_previous_hidden_layer_value(previous_hidden_layer_value)
        self.hidden_layer_values.append(copy.deepcopy(previous_hidden_layer_value))

In [259]:
class output_layer(network_layer):
    
    def forward(self, hidden_layer_output, W_output):
        net_output = np.dot(hidden_layer_output, W_output)
        sigmoid = sigmoid_activation()
        return sigmoid.forward(net_output)

In [261]:
class weight:
    
    @staticmethod
    def GetWeightMatrix(first_dimension, second_dimension):
        return 2*np.random.random((first_dimension,second_dimension)) - 1

**Mean squared error function**

https://en.wikipedia.org/wiki/Mean_squared_error

In [262]:
class loss_function():
    
    @staticmethod
    def mse(target_value, predicted_value):
        return np.mean((target_value - predicted_value)**2)

In [263]:
class utility:
    
    @staticmethod
    def print_result(overallError, a_int, b_int, c, d):    
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

# Train

The general algorithm is

   1. First, present the input pattern and propagate it through the network to get the output.
    
   2. Then compare the predicted output to the expected output and calculate the error.

   3. Then calculate the derivates of the error with respect to the network weights
    
   4. Try to adjust the weights so that the error is minimum.

## Forward propagation

This is our process to predict summation of two bits in each step:

**input layer**

X = two input bits (a,b)

$ net_{input} = X \times  W_{input} $

No activation function is used in this layer

**hidden layer**

$ net_{hidden} = net_{input} + prev_{hidden} \times W_{hidden} $

activation function, which is used in this layer, is sigmoid

$ A_{hidden} = A(net_{hidden}) = \sigma(net_{hidden}) $

**output layer**

$ net_{output} = A_{hidden} \times  W_{output} $

$ \hat{y}\ (predited\ value) = A_{output} = A(net_{output}) = \sigma(net_{output}) $

predited_value = one bit used for the output of the RNN (a+b)


## Back propagation throw time (BPTT)

BPTT works by unrolling all input timesteps. Each timestep has one input time step, one output time step and one copy of the network. Then the errors are calculated and accumulated for each timestep. The network is then rolled back to update the weights.

As can be seen in forward phase, three weights, following weights, are used to compute predicted value, therefore, these weights should be updated for next iteration.

$ W\_output $

$ W\_hidden $

$ W\_input $

### Chain rule

By applying BPTT, they can be updated. In this algorithm, **chain rule** is utilized.

mean squared error is used for cost function:

$ E = \frac{1}{2}(y - \hat{y})^2 $

**$ W_{output} $**

$$ \frac{\partial E}{\partial W_{output}} = \frac{\partial E}{\partial \hat{y}} \times \frac{\partial \hat{y}}{\partial net_{output}} \times \frac{\partial net_{output}}{\partial w_{output}} $$

To compute equation above, following computation are needed:

$ \frac{\partial E}{\partial \hat{y}} = \frac{\partial \frac{1}{2}(y - \hat{y})^2 }{\partial \hat{y}} = 2 \times \frac{1}{2} \times -1 \times (y - \hat{y}) = -(y - \hat{y}) $

$ \frac{\partial \hat{y}}{\partial net_{output}} = \frac{\partial \ \partial (net_{output})} {\partial net_{output}} =  \sigma (net_{output}) (1-\sigma (net_{output})) = \hat{y}(1-\hat{y})$

$ \frac{\partial net_{output}}{\partial w_{output}} =  \frac{\partial A_{hidden} \times  W_{output}}{\partial w_{output}} = A_{hidden}$

By putting these results into main equation, we end up with following equation:

$$ \frac{\partial E}{\partial W_{output}} = -(y - \hat{y}) \times \hat{y}(1-\hat{y}) \times A_{hidden} $$

**$ W_{hidden} $**

$$ \frac{\partial E}{\partial W_{hidden}} = \frac{\partial E}{\partial \hat{y}} \times \frac{\partial \hat{y}}{\partial net_{output}} \times \frac{\partial net_{output}}{\partial A_{hidden}} \times \frac{\partial A_{hidden}}{\partial net_{hidden}} \times \frac{\partial net_{hidden}}{\partial W_{hidden}} $$

To compute equation above, following computation are needed:

$ \frac{\partial E}{\partial \hat{y}} = -(y - \hat{y}) $

$ \frac{\partial \hat{y}}{\partial net_{output}} = \hat{y}(1-\hat{y})$

$ \frac{\partial net_{output}}{\partial A_{hidden}} =  \frac{\partial A_{hidden} \times  W_{output}}{\partial  A_{hidden}} = w_{output}$

$ \frac{\partial A_{hidden}}{\partial net_{hidden}} = \frac{\partial \sigma (net_{hidden})} {\partial net_{hidden}} =  \sigma (net_{hidden}) (1-\sigma (net_{hidden})) = A_{hidden}(1-A_{hidden})$

$ \frac{\partial net_{hidden}}{\partial w_{hidden}} =  \frac{\partial \ (net_{input} + prev_{hidden} \times W_{hidden})}{\partial w_{hidden}} = prev_{hidden} + \frac{\partial prev_{hidden}}{\partial w_{hidden}}\times w_{hidden}$

By putting these results into the main equation, we end up with following equation:

$$ \frac{\partial E}{\partial W_{hidden}} = -(y - \hat{y}) \times \hat{y}(1-\hat{y}) \times w_{output} \times A_{hidden}(1-A_{hidden}) \times (prev_{hidden} + \frac{\partial prev_{hidden}}{\partial w_{hidden}}\times w_{hidden})$$

**$ W_{input} $**

$$ \frac{\partial E}{\partial W_{input}} = \frac{\partial E}{\partial \hat{y}} \times \frac{\partial \hat{y}}{\partial net_{output}} \times \frac{\partial net_{output}}{\partial A_{hidden}} \times \frac{\partial A_{hidden}}{\partial net_{hidden}} \times \frac{\partial net_{hidden}}{\partial net_{input}} \times \frac{\partial net_{input}}{\partial W_{input}}$$

To compute equation above, following computation are needed:

$ \frac{\partial E}{\partial \hat{y}} = -(y - \hat{y}) $

$ \frac{\partial \hat{y}}{\partial net_{output}} = \hat{y}(1-\hat{y})$

$ \frac{\partial net_{output}}{\partial A_{hidden}} = w_{output}$

$ \frac{\partial A_{hidden}}{\partial net_{hidden}} = A_{hidden}(1-A_{hidden})$

$ \frac{\partial net_{hidden}}{\partial net_{input}} =  \frac{\partial \ (net_{input} + prev_{hidden} \times W_{hidden})}{\partial net_{input}} = 1 $

$ \frac{\partial net_{input}}{\partial W_{input}} = \frac{\partial X \times  W_{input}}{\partial W_{input}} = X $

By putting these results into main equation, we end up with following equation:

$$ \frac{\partial E}{\partial W_{input}} = -(y - \hat{y}) \times \hat{y}(1-\hat{y}) \times w_{output} \times A_{hidden}(1-A_{hidden}) \times X $$

In [264]:
class simple_binary_addition_rnn:
    
    def __init__(self, binary_dim, hidden_dimension, learning_rate):
        
        self.binary_dim = binary_dim
        input_dimension = 2
        output_dimension = 1    
        
        # layers
        self.input_layer = input_layer(input_dimension)
        self.hidden_layer = hidden_layer(hidden_dimension)
        self.output_layer = output_layer(output_dimension)
        
        # initialize weights
        self.W_input = weight.GetWeightMatrix(input_dimension, hidden_dimension)
        self.W_hidden = weight.GetWeightMatrix(hidden_dimension, hidden_dimension)
        self.W_output = weight.GetWeightMatrix(hidden_dimension, output_dimension)
        
        self.learning_rate = learning_rate
        self.overallError = 0
        
        # update values for weights
        self.output_layer_deltas = list()
        
    def feed_forward(self, a, b, c):
        
         # Array to save predicted outputs (binary encoded)
        d = np.zeros_like(c)
    
        # position: location of the bit amongst binary_dim-1 bits; for example, starting point "0"; "0 - 7"
        for position in range(binary_dim):

            location = binary_dim - position - 1
            X = np.array([[a[location], b[location]]])

            # Actual value for (a+b) = c, c is an array of 8 bits, so take transpose to compare bit by bit with X value.        
            target = np.array([[c[location]]]).T            
            
            # ----------- forward ---------------
            # input_layer forward
            input_layer_output = self.input_layer.forward(X, self.W_input)
            
            
            # hidden_layer forward
            hidden_layer_output = self.hidden_layer.forward(input_layer_output, self.W_hidden)
            #net_1 = np.dot(X,W_in) + np.dot(hidden_layer_values[-1],W_h) 
            
            # self.output_layer.forward
            # predicated_value is a "guess" for each input matrix. 
            # We can now compare how well it did by subtracting the true answer (y) from the guess (predicated_value). 
            # output_error is just a vector of positive and negative numbers reflecting how much the network missed.
            predicated_value = self.output_layer.forward(hidden_layer_output, self.W_output)            
        
            # Save the hidden layer to be used later            
            #hidden_layer_values.append(copy.deepcopy(layer_1_output))             
            # ToDo
            self.hidden_layer.save_previous_hidden_layer_value(predicated_value)

            # Round off the values to nearest "0" or "1" and save it to a list
            d[location] = np.round(predicated_value[0][0])   
            
        return d
    
    
    
    
    def train(self, epochs_count):
        
        data = dataset(binary_dim)
        
        # This for loop "iterates" multiple times over the training code to optimize our network to the dataset.
        for epoch in range(epochs_count):
            
            # sample a + b = c
            # for example: 2 + 3 = 5 => (a) 00000010 + (b) 00000011 = (c) 00000101
            a, b, c, a_int, b_int, c_int = data.get_sample_addition_problem()
            
            # where we'll store our best guess (binary encoded)
            # desired predictions => d
            d = np.zeros_like(c)  
            
            d = self.feed_forward(a, b ,c)
            
            #back_propagating(a, b)
    
            # Print out the Progress of the RNN
            if (epoch % 10 == 0):
                print_result(overallError, a_int, b_int, c, d)
                
    
    
        

In [245]:
binary_dim = 8
hidden_dimension = 16
learning_rate = 0.1
rnn = simple_binary_addition_rnn(binary_dim, hidden_dimension, learning_rate)